In [ ]:
# Load Modules
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import numpy as np
import pandas as pd
import math
from sklearn import preprocessing

In [ ]:
# Load the dataset
qin_est = pd.read_csv("~/Metagenomics/qin.2014.csv")
qin_est = pd.DataFrame(qin_est)
qin_est.head(10)

In [ ]:
# predictors and response variable
x_qin = qin_est[qin_est.columns[:-1]]
y_qin = qin_est['study_condition']
x_qin.head(10)

In [ ]:
# SVM
# Set up possible values of parameters to optimize over
p_grid = {"C": 2 ** np.arange(-5, 15, 2, dtype = float),
          "gamma": 2 ** np.arange(-15, 3, 2, dtype = float)}

# We will use a Support Vector Classifier with "rbf" kernel
svm = SVC(kernel="rbf")

# Number of random trials
NUM_TRIALS = 20

# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

    
for i in range(NUM_TRIALS):
    
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=inner_cv, scoring='roc_auc')
    clf.fit(x_qin, y_qin)
    non_nested_scores = clf.best_score_
    
    nested_score = cross_val_score(clf, X=x_qin, y=y_qin, cv=outer_cv ,scoring='roc_auc')
    nested_scores[i] = nested_score.mean()

avg_score = nested_scores.mean()

In [ ]:
# Plot scores on each trial for nested CV
# https://matplotlib.org/tutorials/intermediate/legend_guide.html
plt.figure()
plt.subplot(211)
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
# Place a legend to the right of the subplot
plt.legend([nested_line], ["Nested CV"], bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("Nested Cross Validation of SVM after 20 Independent Runs on Cirrhosis Dataset", x=.5, y=1.1, fontsize="15")

In [ ]:
print(avg_score)

In [ ]:
feat = math.floor(math.sqrt(len(x_qin.columns)))
print(feat)

In [ ]:
# Random Forest
p_grid_rf = { # 1st param grid, corresponding to RandomForestRegressor
                            'n_estimators': [500],
                            'max_features': [math.floor(math.sqrt(len(x_qin.columns)))],
                            'citerion': ['gini']}

# Number of random trials
NUM_TRIALS = 20

# Arrays to store scores
rf_scores = np.zeros(NUM_TRIALS)

for i in range(NUM_TRIALS):
    
    clf = RandomForestClassifier(n_estimators=500, criterion='gini', max_features=feat)
    scores = cross_val_score(clf, x_qin, y_qin, cv=10, scoring='roc_auc')
    rf_scores[i] = scores.mean()

print("AUC: %0.2f" % (scores.mean()))
print("Average AUC: %0.2f" % (rf_scores.mean()))

In [ ]:
# Train-test split for the hyperparameter tuning
X_train, x_test, Y_train, y_test = train_test_split(x_qin, y_qin,
                                                    test_size = 0.1, random_state = 0)

In [ ]:
# Feature Importance RF
rf = RandomForestClassifier(n_estimators=500, criterion='gini', max_features=feat)
rf.fit(X_train, Y_train)

In [ ]:
# Get numerical feature importances
print(rf.feature_importances_)

In [ ]:
k = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200]
auc_scores = np.zeros(len(k))
for i in range(len(k)):
    X_feat_train = X_train.iloc[:, rf.feature_importances_.argsort()[-k[i]:]]

    rf_feat = RandomForestClassifier(n_estimators=500, criterion='gini', max_features=math.floor(math.sqrt(k[i])))
    scores = cross_val_score(rf_feat, X_feat_train, Y_train, cv=5, scoring='roc_auc')
    auc_scores[i] = scores.mean()

In [ ]:
# find the number of features that gives that best auc
print(auc_scores)
auc_scores.argsort()[-1:]

In [ ]:
best_feat = int(auc_scores.argsort()[-1:])

In [ ]:
# Random Forest after feature importance

# Number of random trials
NUM_TRIALS = 20

# Arrays to store scores
rf_feat_scores = np.zeros(NUM_TRIALS)

# Subset the original features dataset to include only the features selected in previous step
x_qin_feat = x_qin.iloc[:, rf.feature_importances_.argsort()[-k[best_feat]:]]

for i in range(NUM_TRIALS):
    
    clf = RandomForestClassifier(n_estimators=500, criterion='gini', 
                                 max_features=math.floor(math.sqrt(len(x_qin_feat.columns))))
    scores = cross_val_score(clf, x_qin_feat, y_qin, cv=10, scoring='roc_auc')
    rf_feat_scores[i] = scores.mean()
    
print("AUC: %0.2f" % (scores.mean()))
print("Average AUC: %0.2f" % (rf_feat_scores.mean()))

In [ ]:
print(scores.mean().round(5))
print(rf_feat_scores.mean().round(5))

In [ ]:
# Lasso, can skip

# Set up possible values of parameters to optimize over
alpha = 10 ** np.arange(-4, -0.5, 0.5, dtype = float)

lasso = LassoCV(cv=5, random_state=0, alphas=alpha, tol=0.08)
le = preprocessing.LabelEncoder()
Y_train = le.fit_transform(Y_train)
lasso.fit(X_train, Y_train)


In [ ]:
lasso.score(X_train, Y_train)

In [ ]:
# Lasso
alpha = 10 ** np.arange(-4, -0.5, 0.5, dtype = float)

# Number of random trials
NUM_TRIALS = 20

le = preprocessing.LabelEncoder()
y_qin = le.fit_transform(y_qin)

# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

for i in range(NUM_TRIALS):
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=i)

    lasso = LassoCV(cv=inner_cv, random_state=i, alphas=alpha, tol=0.08).fit(x_qin, y_qin)
    non_nested_scores = lasso.alpha_

    nested_score = cross_val_score(lasso, X=x_qin, y=y_qin, cv=outer_cv, scoring='roc_auc')
    nested_scores[i] = nested_score.mean()

print("AUC: %0.2f" % (nested_score.mean()))
print("Average AUC: %0.2f" % (nested_scores.mean()))

In [ ]:
# Set up possible values of parameters to optimize over
alpha = 10 ** np.arange(-4, -0.5, 0.5, dtype = float)

lasso = LassoCV(cv=5, random_state=0, alphas=alpha, tol=0.08)
le = preprocessing.LabelEncoder()
Y_train = le.fit_transform(Y_train)
sel_ = SelectFromModel(lasso)
sel_.fit(X_train, Y_train)

In [ ]:
print(len(sel_.get_support()))
print(len(x_qin.columns))
sel_.transform(X_train).shape[1]

In [ ]:
x_selected_feat = x_qin.iloc[:, sel_.get_support(indices=True)]

In [ ]:
# Random Forest after feature importance by Lasso

# Number of random trials
NUM_TRIALS = 20

# Arrays to store scores
rf_feat_scores = np.zeros(NUM_TRIALS)

for i in range(NUM_TRIALS):
    
    clf = RandomForestClassifier(n_estimators=500, criterion='gini', 
                                 max_features=math.floor(math.sqrt(len(x_selected_feat.columns))))
    scores = cross_val_score(clf, x_selected_feat, y_qin, cv=10, scoring='roc_auc')
    rf_feat_scores[i] = scores.mean()
    
print("AUC: %0.2f" % (scores.mean()))
print("Average AUC: %0.2f" % (rf_feat_scores.mean()))


In [ ]:
rf_feat_scores.mean()

In [ ]:
# ENet
alpha = 10 ** np.arange(-4, -0.5, 0.5, dtype = float)
l1_ratio = np.array([0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0])

# Number of random trials
NUM_TRIALS = 20

le = preprocessing.LabelEncoder()
y_qin = le.fit_transform(y_qin)

# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

for i in range(NUM_TRIALS):
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=i)

    enet = ElasticNetCV(cv=inner_cv, random_state=i, tol=0.08,
                                l1_ratio=l1_ratio, alphas=alpha).fit(x_qin, y_qin)

    nested_score = cross_val_score(lasso, X=x_qin, y=y_qin, cv=outer_cv, scoring='roc_auc')
    nested_scores[i] = nested_score.mean()

print("AUC: %0.2f" % (nested_score.mean()))
print("Average AUC: %0.2f" % (nested_scores.mean()))

In [ ]:
ElasticNetCV(cv=inner_cv, random_state=0, tol=0.08,
                                l1_ratio=l1_ratio, alphas=alpha).fit(x_qin, y_qin).score(x_qin, y_qin)

In [ ]:
# Set up possible values of parameters to optimize over
alpha = 10 ** np.arange(-4, -0.5, 0.5, dtype = float)
l1_ratio = np.array([0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0])

enet = ElasticNetCV(cv=inner_cv, random_state=i, tol=0.08,
                                l1_ratio=l1_ratio, alphas=alpha)

le = preprocessing.LabelEncoder()
Y_train = le.fit_transform(Y_train)
sel_ = SelectFromModel(lasso)
sel_.fit(X_train, Y_train)

In [ ]:
print(len(sel_.get_support()))
print(len(x_qin.columns))
sel_.transform(X_train).shape[1]

In [ ]:
x_selected_feat = x_qin.iloc[:, sel_.get_support(indices=True)]

In [ ]:
# Random Forest after feature importance by ENet

# Number of random trials
NUM_TRIALS = 20

# Arrays to store scores
rf_feat_scores = np.zeros(NUM_TRIALS)

for i in range(NUM_TRIALS):
    
    clf = RandomForestClassifier(n_estimators=500, criterion='gini', 
                                 max_features=math.floor(math.sqrt(len(x_selected_feat.columns))))
    scores = cross_val_score(clf, x_selected_feat, y_qin, cv=10, scoring='roc_auc')
    rf_feat_scores[i] = scores.mean()
    
print("AUC: %0.2f" % (scores.mean()))
print("Average AUC: %0.2f" % (rf_feat_scores.mean()))

rf_feat_scores.mean()